In [5]:
from src.routing import create_data, get_options_df, solve, prelim_check
import pandas as pd

# Input

In [6]:
# time margin for arrival before appointments
margin = 10 

percentage_of_appointments = 0
GlobalSpanCostCoefficient = 0
slack = 200000
min_work_days = 7

work_schedule = {
    1: [8 * 60, 19 * 60],
    2: [8 * 60, 19 * 60],
    3: [8 * 60, 19 * 60],
    4: [8 * 60, 19 * 60],
    5: [8 * 60, 18 * 60],
    6: [8 * 60, 18 * 60],
    7: [8 * 60, 18 * 60]
}
max_days_off = 0
days_off = {5,6,7,4,3,2}
max_overnight_stays = 2
no_overnight_stays = {}
if len(no_overnight_stays) > 0:
    possible_overnight_stays = set(range(1, 8)) - no_overnight_stays
else:
    possible_overnight_stays = set(range(1, 8))
max_short_days = 0
short_days = {day for day, hours in work_schedule.items() if (hours[1] - hours[0]) / 60 <= 4}

num_nodes = 100

penalty_factor = [10, 10000, 300]
overnight_cost = 2
nodes_df, time_matrix = create_data(min_work_days, days_off, percentage_of_appointments, num_nodes, penalty_factor)
nodes_df, messages, days_off, short_days, max_days_off, max_short_days = prelim_check(nodes_df, time_matrix, work_schedule, days_off, short_days, max_days_off, max_short_days, verbose=False)

total_hours = 0
for day, hours in work_schedule.items():
    if not day in days_off:
        total_hours += hours[1] - hours[0]

n_fixed_appointments = sum(nodes_df['fixed_appointment'].notnull())

avg_spread = time_matrix.iloc[nodes_df['node_id'].values, nodes_df['node_id'].values].median().median()
max_spread = time_matrix.iloc[nodes_df['node_id'].values, nodes_df['node_id'].values].max().max()
# Add: Optimization Algorithms, lunches, lunch duration, etc.

# Throughput

In [7]:
options_df = get_options_df(days_off, max_days_off, short_days, max_short_days, no_overnight_stays, max_overnight_stays)

time_limit = 20
num_nodes_to_consider = 10

nodes_df = nodes_df.sort_values('priority', ascending=False, inplace=False)
top_nodes_df = nodes_df.head(num_nodes_to_consider)
node_zero_df = nodes_df[nodes_df['node_id'] == 0]
nodes_df = pd.concat([node_zero_df, top_nodes_df]).head(num_nodes_to_consider)

# Output

- no fixed appointments -> fixed appointments considered: NaN
- Return absolute number of nodes considered

In [8]:
updated_options_df, figs, route_lists = solve(options_df, nodes_df, time_matrix, slack, work_schedule, margin, time_limit, lunch_duration=30, first_optimization_algorithm='BEST_INSERTION', second_optimization_algorithm='GUIDED_LOCAL_SEARCH', GlobalSpanCostCoefficient=GlobalSpanCostCoefficient, verbose=False, visual=False, restrictive=False)
max_app_cons = updated_options_df['fixed_app_cons'].max()
best_solutions = updated_options_df[updated_options_df['fixed_app_cons'] == max_app_cons]
best_solutions['num_nodes_cons'] -= best_solutions['n_overnight_trips'] * overnight_cost
best_solution = best_solutions['num_nodes_cons'].idxmax()
best_option_df = updated_options_df.iloc[best_solution]
# make values of optimization df
n_overnight_trips = best_option_df['n_overnight_trips']
nodes_cons = best_option_df['nodes_cons']
fixed_app_cons = best_option_df['fixed_app_cons']
print(f'Number of overnight trips: {n_overnight_trips}')
print(f'Number of nodes considered: {nodes_cons}')
print(f'Number of fixed appointments considered: {fixed_app_cons}')

Number of overnight trips: 0
Number of nodes considered: 0.5
Number of fixed appointments considered: 1.0
